## Redemption Fee Floor

The Liquity V2 protocol implements a fixed minimum redemption fee, known as the `REDEMPTION_FEE_FLOOR`, to stop users from exploiting small inaccuracies in oracle-reported prices arising from deviation thresholds. These inaccuracies, called *oracle drift*, typically hover around 0.5% and reflect the difference between an oracle’s price and the true market price. Without this fee, a user could buy ebUSD for $\$ 1$ on the secondary market and redeem it for up to $\$ 1.005$ worth of collateral if the oracle undervalues the asset, profiting from the system. The redemption fee floor, set according to each oracle’s configuration, eliminates this exploit and maintains system stability. When multiple oracles are used, these errors can compound, requiring careful adjustment of the fee based on the asset and its oracle setup.

### Compounding Effect with Multiple Oracles

When an asset’s price is derived from two oracles (e.g., LBTC/BTC and BTC/USD), the maximum deviation—or total error—grows due to a compounding effect. This total error is calculated using the formula:

$$
e_{total} = e_1 + e_2 + e_1 \cdot e_2
$$

Here, $e_1$ is the deviation threshold of the first oracle, and $e_2$ is that of the second oracle. These values represent the potential error in each oracle’s price feed.

### Oracle Configurations and Maximum Errors

Below is a table listing various oracles, their providers, addresses, and deviation thresholds:

| Oracle     | Provider  | Address                                    | Deviation Threshold |
|------------|-----------|--------------------------------------------|---------------------|
| sUSDe/USD  | Redstone  | 0xb99D174ED06c83588Af997c8859F93E83dD4733f | 0.20%              |
| USDe/USD   | Redstone  | 0xbC5FBcf58CeAEa19D523aBc76515b9AEFb5cfd58 | 0.20%              |
| weETH/USD  | Redstone  | 0xdDb6F90fFb4d3257dd666b69178e5B3c5Bf41136 | 1%                 |
| weETH/ETH  | Redstone  | 0x8751F736E94F6CD167e8C5B97E245680FbD9CC36 | 0.50%              |
| LBTC/BTC   | Redstone  | 0xb415eAA355D8440ac7eCB602D3fb67ccC1f0bc81 | 1%                 |
| BTC/USD    | Redstone  | 0xAB7f623fb2F6fea6601D4350FA0E2290663C28Fc | 0.50%              |
| ETH/USD    | Redstone  | 0x67F6838e58859d612E4ddF04dA396d6DABB66Dc4 | 0.50%              |
| WBTC/USD   | Chainlink | 0xfdFD9C85aD200c506Cf9e21F1FD8dd01932FBB23 | 0.10%              |
| cbBTC/USD  | Chainlink | 0x2665701293fCbEB223D11A08D826563EDcCE423A | 2%                 |

There’s also an alternative LBTC oracle from Chainlink with a lower deviation threshold for LBTC/BTC (0.50% vs. 1%), reducing the total error:

| Oracle    | Provider  | Address                                    | Deviation Threshold |
|-----------|-----------|--------------------------------------------|---------------------|
| LBTC/BTC  | Chainlink | 0x5c29868C58b6e15e2b962943278969Ab6a7D3212 | 0.50%              |
| BTC/USD   | Chainlink | 0xF4030086522a5bEEa4988F8cA5B36dbC97BeE88c | 0.50%              |

Using these oracles, the maximum error for different assets is calculated as follows, with the recommended redemption fee floor values included:

| Asset           | Oracle Config.        | Max Error | Calculation                          | REDEMPTION_FEE_FLOOR (%) | REDEMPTION_FEE_FLOOR (bps) |
|-----------------|-----------------------|-----------|--------------------------------------|--------------------------|----------------------------|
| sUSDe           | sUSDe/USD             | 0.20%     | Direct oracle                        | 0.50%                    | 50 bps                     |
| weETH           | weETH/USD             | 1%        | Direct oracle                        | 1%                       | 100 bps                    |
| weETH           | weETH/ETH > ETH/USD   | 1.025%    | 0.5% + 0.5% + (0.5% × 0.5%) = 1.025% | 1.10%                    | 110 bps                    |
| WBTC            | WBTC/USD              | 0.10%     | Direct oracle                        | 0.50%                    | 50 bps                     |
| cbBTC           | cbBTC/USD             | 2%        | Direct oracle                        | 2%                       | 200 bps                    |
| LBTC (Redstone) | LBTC/BTC > BTC/USD    | 2%        | 1% + 0.5% + (1% × 0.5%) = 2%        | 2%                       | 200 bps                    |
| LBTC (Chainlink)| LBTC/BTC > BTC/USD    | 1.025%    | 0.5% + 0.5% + (0.5% × 0.5%) = 1.025% | 1.10%                    | 110 bps                    |


In the Liquity V2 protocol, the redemption fee is not static but adjusts dynamically based on redemption activity. At a given time $t$ and redemption event $j$ , the redemption fee $f(t_j)$ can be expressed as:

$$
f(t_j) = f_{min} + b(t_j)
$$

where $f_{min}$ is `REDEMPTION_FEE_FLOOR`, and $b(t_j)$ is a dynamic fee that scales linearly with repspect to the redeemed amount $m$ relative to the total supply $n$, subject to a scaling parameter $\alpha$. This formulation is detailed in [Chaos Labs Liquity V2 Mechanism Design Review](https://cdn.sanity.io/files/zmh9mnff/production/ca6a4815e62b05f33fb3ec56c5a4c42d6b7ddbec.pdf).

The structure of this fee implies that large redemption sizes are less likely to reflect attempts to exploit oracle inaccuracies (oracle drift). Conversely, numerous small redemptions, potentially originating from one or a few addresses, could suggest a "skimming off the top" strategy, where users repeatedly redeem small amounts to profit from minor price discrepancies between the oracle-reported price and the actual market price. Monitoring multiple small redemptions from a single address (or multiple) may reveal attempts of gaming the system.

# Critical System Collateral Ratio (CCR)

As an asset’s price declines in a lending system, pushing positions closer to liquidation, the Total Collateral Ratio (TCR),the ratio of collateral value ($C$) to borrowed amount ($B$), decreases:

$$
TCR = \frac{C}{B}
$$

A related metric, the Health Factor (HF), gauges position safety. For an individual position, $HF = \frac{C_i}{B_i \times MCR}$, where $MCR$ is the Minimum Collateralization Ratio. For an entire branch, the average HF is:

$$
HF = \frac{TCR}{MCR}
$$

Positions with $HF < 1.2$ are typically at risk of liquidation. If a large portion borrowed value in a branch is at risk, the system has automatic measures in place to pause borrowing and limit collateral withdrawals, allowing them only with debt repayment equal to or exceeding the withdrawn collateral.

The Critical Collateral Ratio (CCR) is the TCR threshold where protective measures kick in. When $TCR = CCR$, the average $HF$ becomes:

$$
HF = \frac{CCR}{MCR}
$$

Liquity V2, for example, sets:
- **WETH**: $MCR = 110\%$, $CCR = 150\%$, $HF = \frac{1.5}{1.1} \approx 1.36$
- **sETH**: $MCR = 120\%$, $CCR = 160\%$, $HF = \frac{1.6}{1.2} \approx 1.33$

To align with Liquity V2’s standards (targeting $HF = 1.2–1.4$), borrowing should be restricted when average HF falls into this range.  
Using Loan-to-Value (LTV) ranges, we convert to MCR ($MCR = \frac{100}{LTV}$) and calculate CCR to keep $HF$ between 1.2 and 1.4:

### Step 1: Convert LTV to MCR
First, let's convert the LTV values to MCR ($MCR = \frac{100}{LTV}$).

| Asset  | LTV Range | MCR (Rounded) |
|--------|-----------|---------------|
| sUSDe  | 88-84-80% | 114-119-125% |
| weETH  | 78-74-70% | 128-135-143% |
| WBTC   | 76-72-68% | 132-139-147% |
| cbBTC  | 76-72-68% | 132-139-147% |
| LBTC   | 74-70-66% | 135-143-152% |

---

### Step 2: Calculate CCR Using the Health Factor Range
From the previous section, the Health Factor (HF) at the point where $TCR = CCR$ is:

$$
HF = \frac{CCR}{MCR}
$$

We want $HF$ to be within 1.2 to 1.4. Therefore, for each $MCR$, we calculate $CCR$ as:

- $CCR_{min} = MCR \times 1.2$ (for $HF = 1.2$)
- $CCR_{max} = MCR \times 1.4$ (for $HF = 1.4$)

We’ll compute these for each asset’s MCR range and select a single $CCR$ that ensures $HF$ stays within 1.2 to 1.4 across the entire MCR range.

#### sUSDe
- **MCR = 114%**:
  - $CCR_{min} = 1.14 \times 1.2 = 1.368$
  - $CCR_{max} = 1.14 \times 1.4 = 1.596$
- **MCR = 119%**:
  - $CCR_{min} = 1.19 \times 1.2 = 1.428$
  - $CCR_{max} = 1.19 \times 1.4 = 1.666$
- **MCR = 125%**:
  - $CCR_{min} = 1.25 \times 1.2 = 1.5$
  - $CCR_{max} = 1.25 \times 1.4 = 1.75$

To keep $HF$ within 1.2 to 1.4 across all MCRs:
- At $MCR = 114$, $CCR$ must be at least 136.8% ($HF \geq 1.2$).
- At $MCR = 125$, $CCR$ must not exceed 175% ($HF \leq 1.4$).
- Choose $CCR = 150\%$:
  - $MCR = 114$: $HF = \frac{1.5}{1.14} \approx 1.316$
  - $MCR = 119$: $HF = \frac{1.5}{1.19} \approx 1.261$
  - $MCR = 125$: $HF = \frac{1.5}{1.25} = 1.2$

#### weETH
- **MCR = 128%**:
  - $CCR_{min} = 1.28 \times 1.2 = 1.536$
  - $CCR_{max} = 1.28 \times 1.4 = 1.792$
- **MCR = 135%**:
  - $CCR_{min} = 1.35 \times 1.2 = 1.62$
  - $CCR_{max} = 1.35 \times 1.4 = 1.89$
- **MCR = 143%**:
  - $CCR_{min} = 1.43 \times 1.2 = 1.716$
  - $CCR_{max} = 1.43 \times 1.4 = 2.002$

Choose $CCR = 170\%$ (keeps $HF$ in range):
- $MCR = 128$: $HF = \frac{1.7}{1.28} \approx 1.328$
- $MCR = 135$: $HF = \frac{1.7}{1.35} \approx 1.259$
- $MCR = 143$: $HF = \frac{1.7}{1.43} \approx 1.189$ (slightly below 1.2)

#### WBTC
- **MCR = 132%**:
  - $CCR_{min} = 1.32 \times 1.2 = 1.584$
  - $CCR_{max} = 1.32 \times 1.4 = 1.848$
- **MCR = 139%**:
  - $CCR_{min} = 1.39 \times 1.2 = 1.668$
  - $CCR_{max} = 1.39 \times 1.4 = 1.946$
- **MCR = 147%**:
  - $CCR_{min} = 1.47 \times 1.2 = 1.764$
  - $CCR_{max} = 1.47 \times 1.4 = 2.058$

Choose $CCR = 175\%$:
- $MCR = 132$: $HF = \frac{1.75}{1.32} \approx 1.326$
- $MCR = 139$: $HF = \frac{1.75}{1.39} \approx 1.259$
- $MCR = 147$: $HF = \frac{1.75}{1.47} \approx 1.190$

#### cbBTC
- Same as WBTC: $CCR = 175\%$, with $HF$ values of 1.326, 1.259, and 1.190.

#### LBTC
- **MCR = 135%**:
  - $CCR_{min} = 1.35 \times 1.2 = 1.62$
  - $CCR_{max} = 1.35 \times 1.4 = 1.89$
- **MCR = 143%**:
  - $CCR_{min} = 1.43 \times 1.2 = 1.716$
  - $CCR_{max} = 1.43 \times 1.4 = 2.002$
- **MCR = 152%**:
  - $CCR_{min} = 1.52 \times 1.2 = 1.824$ 
  - $CCR_{max} = 1.52 \times 1.4 = 2.128$

Choose $CCR = 180\%$:
- $MCR = 135$: $HF = \frac{1.8}{1.35} \approx 1.333$
- $MCR = 143$: $HF = \frac{1.8}{1.43} \approx 1.259$
- $MCR = 152$: $HF = \frac{1.8}{1.52} \approx 1.184$ (slightly below 1.2)

---

### Final Results
Here’s a table summarizing the $CCR$ values and the resulting $HF$ range for each asset:

| Asset  | LTV Range | MCR Range (%) | CCR (%)  | HF Range       |
|--------|-----------|-----------|------|----------------|
| sUSDe  | 88–80     | 114–125  | 150 | 1.20–1.316     |
| weETH  | 78–70     | 128–143  | 170 | 1.189–1.328    |
| WBTC   | 76–68     | 132–147  | 175 | 1.190–1.326    |
| cbBTC  | 76–68     | 132–147  | 175 | 1.190–1.326    |
| LBTC   | 74–66     | 135–152  | 180 | 1.184–1.333    |

### Notes
- Some $HF$ values dip slightly below 1.2 at the higher end of the MCR range, but they remain close to the target.
- The chosen $CCR$ values are rounded to whole numbers for practicality.
- These CCR values give leeway for setting LTVs higher by ~4% past the upper range while maintaining trigger at $HF \leq 1.4$

# Shutdown System Collateral Ratio (SCR)
The Shutdown System Collateral Ratio (SCR) is a key safety threshold. When the Total Collateral Ratio (TCR) falls below SCR, the protocol halts all borrowing except for position closures, triggering a market shutdown. Users can then redeem ebUSD for collateral at a rate better than the current oracle price. This protects the system by curbing the creation of new debt and promoting deleveraging when most positions drop below the branch’s Minimum Collateralization Ratio (MCR). Think of SCR as a last line of defense in a worst-case scenario.

To achieve this, SCR should be set close to the MCR so that the shutdown triggers when most positions are at or below this minimum threshold, prompting forced liquidations to limit potential losses for the protocol. A key consideration is that SCR is immutable, once set it cannot be adjusted after deployment. To future-proof the system, SCR should not exceed the lowest conceivable MCR the protocol might adopt in the future. Keeping consistent with CCR, lets add 4 percentage points to the upper end of each asset's LTV range and set SCR based on this.

| Asset  | Theoretical Max LTV | Corresponsing Min MCR (rounded %) | SCR Recommendation (%)  |
|--------|-----------|-----------|------|
| sUSDe  | 92     | 109  | 110 |
| weETH  | 82     | 122  | 122 |
| WBTC   | 80     | 125  | 125 |
| cbBTC  | 80     | 125  | 125 |
| LBTC   | 78     | 128  | 128 |